In [1]:
import requests
import pandas as pd
import numpy as np
import time

In [2]:
def fetch_agg_trades(symbol, start_time, end_time=None, limit=1000):
    url = "https://api.binance.com/api/v3/aggTrades"
    trades = []
    from_id = None

    while True:
        params = {
            "symbol": symbol,
            "limit": limit
        }

        if from_id is not None:
            params["fromId"] = from_id
        else:
            params["startTime"] = start_time
            if end_time:
                params["endTime"] = end_time

        r = requests.get(url, params=params)
        data = r.json()

        if not data:
            break

        trades.extend(data)
        from_id = data[-1]["a"] + 1  # aggTradeId suivant

        if len(data) < limit:
            break

        time.sleep(0.2)  # respecter l’API

    return trades


In [3]:
symbol = "BTCUSDT"

# exemple : 1 heure
start_time = int(pd.Timestamp("2026-01-02 10:00:00", tz="UTC").timestamp() * 1000)
end_time   = int(pd.Timestamp("2026-01-02 11:00:00", tz="UTC").timestamp() * 1000)

raw_trades = fetch_agg_trades(symbol, start_time, end_time)

df = pd.DataFrame(raw_trades)


In [4]:
df['price'] = df['p'].astype(float)
df['qty'] = df['q'].astype(float)

df['time'] = pd.to_datetime(df['T'], unit='ms', utc=True)
df.set_index('time', inplace=True)


In [5]:
df['delta'] = np.where(
    df['m'] == False,   # buyer NOT maker → buy market
    df['qty'],          # pression acheteuse
    -df['qty']          # pression vendeuse
)

df['cdv'] = df['delta'].cumsum()
cdv_ohlc = df['cdv'].resample('1min').ohlc()


In [ ]:
cdv_ohlc

,open,high,low,close
time,,,,
2026-01-02 10:00:00+00:00,0.00031,47.68171,0.00031,41.90539
2026-01-02 10:01:00+00:00,41.90617,62.32556,41.90617,61.41934
2026-01-02 10:02:00+00:00,61.41843,63.42622,46.38604,46.48815
2026-01-02 10:03:00+00:00,46.56535,53.44850,44.82679,45.71178
2026-01-02 10:04:00+00:00,45.71089,46.13639,44.88407,44.91095
...,...,...,...,...
2026-01-02 17:44:00+00:00,72.90795,73.65643,71.96349,73.63880
2026-01-02 17:45:00+00:00,73.63870,83.46123,73.58865,82.57229
2026-01-02 17:46:00+00:00,82.57236,84.75163,78.05666,79.48232


: 